In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from astropy import constants

## Tips for calculating force derivative in P1

In the project we have N different Solar system bodies each with mass, position, and velocity. Let's initialise some random values to fill these parameters into a state vector $S$. We use positions between $[-100, 100]$ and velocities between $[-10, 10]$ in three dimensions. We use this to create our state vector with dimensions $N\times6$. The columns are $x, y, z, v_x, v_y, v_z$

In [ ]:
rng = np.random.default_rng()

# Number of bodies
N = 4

mass = np.arange(N)
pos = rng.uniform(low=[-100, -100, -100], high=[100, 100, 100], size=(N, 3))
vel = rng.uniform(low=[-10, -10, -10], high=[10, 10, 10], size=(N, 3))

S = np.hstack((pos, vel))
print(S.shape)

Now we want to calculate the derivative of the state vector, $dS$. The derivative of the positions is simple, since

$$\frac{d\mathbf{x}_i}{dt} = \mathbf{v_i}$$

So:

In [ ]:
dS = np.zeros_like(S)

dS[:, :3] = S[:, 3:]

The derivative of the velocites are not quite as straight forward. We have

$$\frac{d\mathbf{v}_i}{dt} = -\sum^N_{j=1; j\neq i}\frac{Gm_j(\mathbf{x_i - x_j})}{|\mathbf{x_i - x_j}|^3}$$

Which can be done inefficiently, but simply, through a for-loop. We will however do this in a vectorised way. To do this, we want to calculate the matrix of $\Delta x_{ij}$ of shape $N \times N \times 3$ which contains $\mathbf{x_i - x_j}$ for all combinations of $i$ and $j$ as such:

$$ \begin{bmatrix}
\mathbf{x}_0 - \mathbf{x}_0 & \mathbf{x}_0 - \mathbf{x}_1 & \dots & \mathbf{x}_0 - \mathbf{x}_N \\
\mathbf{x}_1 - \mathbf{x}_0 & \mathbf{x}_1 - \mathbf{x}_1 & \dots & \mathbf{x}_1 - \mathbf{x}_N \\
\vdots & \vdots & \ddots & \vdots \\
\mathbf{x}_N - \mathbf{x}_0 & \mathbf{x}_N - \mathbf{x}_1 & \dots & \mathbf{x}_N - \mathbf{x}_N 
\end{bmatrix}  $$

To do this in numpy, we simply have to make two copies of our position array. One that is $1 \times N \times 3$ and one that is $N \times 1 \times 3$

In [ ]:
pos = S[:, 3:]
print(pos.shape)

In [ ]:
xi = pos[:, np.newaxis]
xj = pos[np.newaxis]
print(xi.shape)
print(xj.shape)

In [ ]:
dxij = xi - xj
print(dxij.shape)

Remember that we still have the self-interaction which lies along the diagonal where $i=j$. We will now calculate the matrix within the sum above. To be able to multiply in m into the sum above, it needs to be the right shape. It needs to be multiplied into each column $j$, so should be a column vector. We achieve this as such: 

In [ ]:
print(mass.shape)
mass = mass[np.newaxis, :, np.newaxis]
print(mass.shape)

Now we calculate

$$\frac{Gm_j(\mathbf{x_i - x_j})}{|\mathbf{x_i - x_j}|^3}$$

In [ ]:
G = constants.G.value
acc_matrix = G * mass * dxij / np.linalg.norm(dxij, axis=2)[..., np.newaxis]

Now you will have probably gotten a warning saying: `RuntimeWarning: invalid value encountered in true_divide`.
This is because we didn't handle the self-interaction. When i=j, the denominator is 0 and we obviously cannot divide by zero. Because of this we get `nan`. Thankfully, now that we are going to sum, numpy provides the handy function `np.nansum`

In [ ]:
?np.nansum

In [ ]:
dS[:, 3:] = -np.nansum(acc_matrix, axis=1)

And so, we have calculated the acceleration and $dS$ without using a single for-loop

# Kernel / NNSA calculations in P2/P3

In the following projects, we deal with the kernel function. 
It's form and derivative is the following:

$$
W(R, h) = \alpha_d
\begin{cases}
  \frac{2}{3} - R^2 + \frac{1}{2}R^3 & 0\leq R <1 \\
  \frac{1}{6}(2-R)^3 & 1\leq R <2 \\
  0 & R \geq 2 \\
\end{cases}
$$

$$
dW(R, h) = \alpha_d
\begin{cases}
  \left(-2 + \frac{3}{2}R\right)\frac{d\mathbf{x}}{h^2} & 0\leq R <1 \\
  -\frac{1}{2}(2-R)^2 \frac{d\mathbf{x}}{hr}& 1\leq R <2 \\
  0 & R \geq 2 \\
\end{cases}
$$

But we also know that

$$R_{ij} = \frac{r_{ij}}{h} = \frac{|d\mathbf{x}|}{h}$$

Which means that our criterias can be written as:

$$0\leq |d\mathbf{x}| <h$$
$$1h\leq |d\mathbf{x}| <2h$$

Let's create 3D positions to use so we can once more calculate $d\mathbf{x} = \Delta \mathbf{x}_{ij}$. To simulate P2, we'll use 400 particles in two regions and use some value for h.

In [ ]:
h = 0.004
N = 400
pos = np.zeros((400, 3))
pos[:320, 0] = np.linspace(-0.6, 0, 320)
pos[320:, 0] = np.linspace(0, 0.6, 80)
print(pos.shape)

In [ ]:
dxij = pos[:, np.newaxis] - pos[np.newaxis]
rij = np.linalg.norm(dxij, axis=2)
R = np.linalg.norm(dxij, axis=2) / h
print(dxij.shape)
print(R.shape)

Now we can create 2 boolean masks to fulfill the different criteria in the kernel

In [ ]:
I1 = (0 <= R) & (R < 1)
I2 = (1 <= R) & (R < 2)
print(I1)

We now have all we need to calculate both $W$ and $dW$

In [ ]:
W = np.zeros_like(rij)
dW = np.zeros_like(dxij)

By masking out the values where `bool_mask_1` is true in $W$, $dW$ we assign only those entries in them. So:

In [ ]:
alpha = 1

W[I1] = alpha * (2 / 3 - R[I1] ** 2 + 1 / 2 * R[I1] ** 3)
W[I2] = alpha * (1 / 6 * (2 - R[I2]) ** 3)

dW[I1] = alpha * (-2 + 3 / 2 * R[I1][:, np.newaxis]) * dxij[I1, :] / h**2
dW[I2] = (
    alpha
    * (-1 / 2 * (2 - R[I2][:, np.newaxis]) ** 2)
    * dxij[I2, :]
    / (h * rij[I2][:, np.newaxis])
)

Now we have filled in the parts of $W$ and $dW$ with the relevant values. The case of $R\geq2$ is handled automatically since we initialise both arrays as zero, and never need to change them.

If you need to see how many neighbours you have for a specific particles, i, you simply call `I1[i]` and count the number of `True` occurences. You can also just sum it:

In [ ]:
sum(I1[0])

Of course this means numpy can gives you the number of neighbors for each particle by summing in the right axis then

In [ ]:
I1.sum(axis=0)

# The dot product of $v_{ij}$ and $x_{ij}$ in P3

In P2/P3, you get to the following equation

$$\mathbf{v}_{ij} \cdot \mathbf{x}_{ij}$$

This is the dot product between the two. In P2 where it is 1D it is simply a multiplication of our two $N \times N$ arrays. When they are 3D however, we want to make sure we get the dot product right.

Let us create a version of $\Delta\mathrm{v}_{ij}$

In [ ]:
dvij = rng.normal(size=dxij.shape)

The dot product of two vectors would simply be the sum of the product, like so (here for partcles ($i, j) = (0, 1)$:

In [ ]:
np.sum(dxij[0, 1] * dvij[0, 1])

We can of course do this for two arrays as well if we simply sum along the right axis

In [ ]:
dxdvij_A = np.sum(dxij * dvij, axis=2)

We can use `numpy.einsum` as well

In [ ]:
dxdvij_B = np.einsum("ijk,ijk->ij", dxij, dvij)

Or work cleverly with the `@` operator which does matrix multiplication

In [ ]:
dxdvij_C = np.squeeze(dxij[..., None, :] @ dvij[..., :, None])

We can check if they are identical:

In [ ]:
np.allclose(dxdvij_A, dxdvij_B), np.allclose(dxdvij_B, dxdvij_C)

As a check we look at particles $(i, j) = (0, 1)$ again

In [ ]:
dxdvij_A[0, 1], dxdvij_B[0, 1], dxdvij_C[0, 1]

And we'll see how they perform:

In [ ]:
%timeit np.sum(dxij * dvij, axis=2)

In [ ]:
%timeit np.einsum('ijk,ijk->ij', dxij, dvij)

In [ ]:
%timeit np.squeeze(dxij[..., None, :] @ dvij[..., :, None])